# Projeto: Dataset MS Marco Traduzido para Português
### Autores : Graziella Cardoso Bonadia e Matheus Gustavo Alves Sasso


In [ ]:
#@title Configurações gerais
experiment_name = 'NaturalQuestionsPortuguese'  #@param {type:"string"}
model_name = 't5-small'  #@param ["t5-base","t5-small","t5-large"] {type:"string"}
TARGET_LANGUAGE = 'portuguese'  #@param {type:"string"}
SOURCE_LANGUAGE = 'english' #@param {type:"string"}
PREFIX = 'translate English to Portuguese: '

# Instalação de pacotes , imports e configurações gerais

### Pacotes Externos

In [ ]:
! pip install pytorch-lightning==0.7.6 --quiet
! pip install transformers --quiet
! pip install nvidia-smi --quiet
! pip install ftfy --quiet
! pip install jsonlines --quiet
! pip install sacrebleu --quiet

### Funções auxiliares 

In [ ]:
%%capture
!wget -nc https://raw.githubusercontent.com/Matheus158257/ms-marco-passage-ranking-dense-vectors-with-doc2query/master/read_ms_marco.py
!wget -nc https://raw.githubusercontent.com/Matheus158257/ms-marco-passage-ranking-dense-vectors-with-doc2query/master/read_ms_marco2.py

### Paths

Root

In [ ]:
import os
# data_base_dir = '/content/drive/My Drive/Mestrado/PLN/Projeto/Data/Traducao/Natural_Questions' #Matheus
data_base_dir = '/content/drive/My Drive/Projeto/Data/Traducao/Natural_Questions'
# data_base_dir = '/content/drive/My Drive/Projeto/Data/Traducao/Natural_Questions' #Graziella
# check_path = '/content/drive/My Drive/Projeto/Data/Traducao/checkpoints'
check_path = '/content/drive/My Drive/Projeto/Data/Traducao'

In [ ]:
# NO CHUNCK
nq_doct_text_path =  data_base_dir + '/nq_doc_text.csv'
nq_long_answer_path =  data_base_dir + '/nq_long_answer.csv'
nq_short_answers_path =  data_base_dir + '/nq_short_answer.csv'
nq_question_path =  data_base_dir + '/nq_question.csv'
nq_infos_path =  data_base_dir + '/nq_infos.csv'

# # CHUNCK
# nq_doct_text_chunk_path =  data_base_dir + '/nq_doc_text_chunk.csv'
# nq_long_answer_chunk_path =  data_base_dir + '/nq_long_answer_chunk.csv'
# nq_short_answers_chunk_path =  data_base_dir + '/nq_short_answer_chunk.csv'
# nq_question_chunk_path =  data_base_dir + '/nq_question_chunk.csv'
# nq_infos_chunk_path =  data_base_dir + '/nq_infos_chunk.csv'

NameError: ignored

### Imports


In [ ]:
#Bibliotecas Padrão
import os
import random
from typing import Dict
from typing import List
from typing import Tuple
import re
import gzip
import math
import jsonlines
import pdb
import ftfy


#Bibliotecas Data Science
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
import numpy as np
import pandas as pd

#Bibliotecas Pytorch
import torch
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils.data import DataLoader
import torch.nn.functional as F

#Bibliotecas Lightning
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

#Bibliotecas transfromers
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer

#import das funções que vem do github
import collections
import functools
import traceback
import sys
import os
from read_ms_marco2 import load_qrels
from read_ms_marco2 import load_queries
from read_ms_marco import load_collection
from read_ms_marco import load_doc2query
from read_ms_marco import load_triple
from read_ms_marco2 import load_txts_topk #load_txts_topk(folder,k=1,n=18,encoding="cp1252")

# Score Metrics
import sacrebleu

# Initialize tensorboard
print("\nInitialize=ing Tensorboard...\n")
%load_ext tensorboard

# Device
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu" 
print(dev)
device = torch.device(dev)


Initialize=ing Tensorboard...

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
cuda:0


### Fix seeds 

In [ ]:
seed = 123
random.seed(seed)
# np.random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)

### Montar o Drive

In [ ]:
#Mount drive
print("\nMounting Drive...\n")
from google.colab import drive
drive.mount('/content/drive')


Mounting Drive...

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Impedir excesso de logs

In [ ]:
import logging
logging.getLogger("transformers.configuration_utils").setLevel(logging.WARNING)
logging.getLogger("transformers.modeling_utils").setLevel(logging.WARNING)
logging.getLogger("lightning").setLevel(logging.WARNING)

###Impedir quebra de memória

In [ ]:
# https://docs.fast.ai/troubleshoot.html#memory-leakage-on-exception
import functools, traceback
def gpu_mem_restore(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = sys.exc_info()
            traceback.clear_frames(tb)
            raise type(val).with_traceback(tb) from None
    return wrapper


###Hardware Data

In [ ]:
import nvidia_smi
import psutil
from multiprocessing import cpu_count
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
print("\nGetting Hardware Statatus...\n")
def hardware_stats():
    '''
    Returns a dict containing some hardware related stats
    '''
    res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
    return {"cpu": str(psutil.cpu_percent()) + '%',
            "mem": str(psutil.virtual_memory().percent) + '%',
            "gpu": str(res.gpu) + '%',
            "gpu_mem": str(res.memory) + '%'}

print(f"Imports loaded succesfully. Current GPU: {torch.cuda.get_device_name(0)}, number of CPU cores: {cpu_count()}")


Getting Hardware Statatus...

Imports loaded succesfully. Current GPU: Tesla K80, number of CPU cores: 2


In [ ]:
print(f"Pytorch Lightning Version: {pl.__version__}")
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
print(f"Device name: {nvidia_smi.nvmlDeviceGetName(handle)}")
print(f"Number of CPU cores: {cpu_count()}")

def gpu_usage():
    global handle
    return str(nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu) + '%'

Pytorch Lightning Version: 0.7.6
Device name: b'Tesla K80'
Number of CPU cores: 2


# Trabalhando o modelo

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
extra_tokens = ['À' , 'È' , 'Ì' , 'Ò' , 'Ù' , 'à' , 'è' , 'ì' , 'ò' , 'ù' , 'Á' , 'É' , 'Í' , 'Ó' , 'Ú' , 'á' , 'é' , 'í' , 'ó' , 'ú' , 'Â' , 'Ê' , 'Î' , 'Ô' , 'Û' , 'â' , 'ê' , 'î' , 'ô' , 'û'  , 'Ã' , 'Õ'  , 'ã', 'õ' , 'Ë', 'ä' , 'ë' , 'ï' , 'ö' , 'ü']

In [ ]:
tokenizer.add_tokens('não')

1

In [ ]:
# tokenizer.add_tokens('<CHUNK>')

Alguns dos tokens de acentuação já existem no dataset. Checamos isso fazendo o encoding do token. Se existir, ele retorna um token x. Se não, retorna 2. Se for 2, a gente adiciona para o tokenizador

In [ ]:
added_tokens = []
for tok in extra_tokens:
    enc = tokenizer.encode(tok)
    if 2 in enc:
        added_tokens.append(tok)
        tokenizer.add_tokens(tok)

In [ ]:
def fix_accent_breaks(text):
    """
    A ideia é fazer a junção de letras com acento de volta para frases na validação.
    Isso serve para melhorar o BLEU.
    
    Args
        text: texto que terá acentuação corrigida
        
    Returns:
        Texto completo com acentuação corrigida
    
    
    
    """
    words = text.split(" ")
    out_words = []
    merge_pos = [idx for idx, dat in enumerate(words) if dat in added_tokens]
    for pos in sorted(merge_pos, reverse=True):
        if pos==0:
            new_word = words[pos]+words[pos+1]
            for i in range(2): words.pop
            words.pop(pos+1)
            words.pop(pos)        
            words.insert(pos, new_word)
        elif pos==len(words)-1:
            new_word = words[pos-1]+words[pos]
            words.pop(pos)
            words.pop(pos-1)
            words.insert(pos-1, new_word)
        else:
            new_word = words[pos-1]+words[pos]+words[pos+1]
            words.pop(pos+1)
            words.pop(pos)
            words.pop(pos-1)
            words.insert(pos-1, new_word)

    return " ".join(words)

# Etapas do paracrawl -> Treino




## Data Prep

In [ ]:
! wget -nc https://storage.googleapis.com/neuralresearcher_data/unicamp/ia376e_2020s1/paracrawl_enpt_train.tsv.gz
! wget -nc https://storage.googleapis.com/neuralresearcher_data/unicamp/ia376e_2020s1/paracrawl_enpt_test.tsv.gz

File ‘paracrawl_enpt_train.tsv.gz’ already there; not retrieving.

File ‘paracrawl_enpt_test.tsv.gz’ already there; not retrieving.



In [ ]:
def load_text_pairs(path):
    text_pairs = []
    for line in gzip.open(path, mode='rt'):
        text_pairs.append(line.strip().split('\t'))
    return text_pairs

ds_train = load_text_pairs('paracrawl_enpt_train.tsv.gz')
ds_test = load_text_pairs('paracrawl_enpt_test.tsv.gz')

# Embaralhamos o treino para depois fazermos a divisão treino/val.
random.shuffle(ds_train)

# Truncamos o dataset para 100k pares de treino e 5k pares de validação.
ds_val = ds_train[100000:105000]
ds_train = ds_train[:100000]

for set_name, x in [('treino', ds_train), ('validação',ds_val), ('test', ds_test)]:
    print(f'\n{len(x)} amostras de {set_name}')
    print(f'3 primeiras amostras {set_name}:')
    for i, (source, target) in enumerate(x[:3]):
        print(f'{i}: source: {source}\n   target: {target}')


100000 amostras de treino
3 primeiras amostras treino:
0: source: More Croatian words and phrases
   target: Mais palavras e frases em croata
1: source: Jerseys and pullovers, containing at least 50Â % by weight of wool and weighing 600Â g or more per article 6110 11 10 (PCE)
   target: Camisolas e pulôveres, com pelo menos 50 %, em peso, de lã e pesando 600g ou mais por unidade 6110 11 10 (PCE)
2: source: Atex Colombia SAS makes available its lead product, 100% natural liquid latex, excellent quality and price. ... Welding manizales caldas Colombia a DuckDuckGo
   target: Atex Colômbia SAS torna principal produto está disponível, látex líquido 100% natural, excelente qualidade e preço. ...

5000 amostras de validação
3 primeiras amostras validação:
0: source: «You have hidden these things from the wise and the learned you have revealed them to the childlike»
   target: «Escondeste estas coisas aos sábios e entendidos e as revelaste aos pequenos»
1: source: Repair of computers, applic

## Classe de dataset

In [ ]:
class ParacrawlDataset(Dataset):
    def __init__(self, text_pairs: List[Tuple[str]], tokenizer,
                 source_max_length: int = 32, target_max_length: int = 32):
        self.tokenizer = tokenizer
        self.text_pairs = text_pairs
        self.source_max_length = source_max_length
        self.target_max_length = target_max_length
        
    def __len__(self):
        return len(self.text_pairs)
    
    def __getitem__(self, idx):
        source, target = self.text_pairs[idx]
        source_modified = 'translate English to Portuguese: '+ source + self.tokenizer.eos_token
        target_modified = target + self.tokenizer.eos_token
        
        source_tok = tokenizer.batch_encode_plus([source_modified], add_special_tokens=True,
                                                max_length=self.source_max_length, pad_to_max_length = True, 
                                                return_tensors='pt')
        target_tok = tokenizer.batch_encode_plus([target_modified], add_special_tokens=True,
                                                max_length=self.target_max_length, pad_to_max_length = True, 
                                                return_tensors='pt')
        
        return (source_tok['input_ids'][0], source_tok['attention_mask'][0], target_tok['input_ids'][0], 
                target_tok['attention_mask'][0], source, target)

In [ ]:
text_pairs = [('Dear friends, the history of this nation includes many examples of the Church’s commitment in this regard.',
               'Queridos amigos, a história desta Nação oferece numerosos exemplos do compromisso da Igreja a este propósito.')]
dataset_debug = ParacrawlDataset(
    text_pairs=text_pairs,
    tokenizer=tokenizer,
    source_max_length=50,
    target_max_length=50)

dataloader_debug = DataLoader(dataset_debug, batch_size=10, shuffle=True, 
                              num_workers=0)

source_token_ids, source_mask, target_token_ids, target_mask, _, _ = next(iter(dataloader_debug))
print('source_tokens:\n', tokenizer.tokenize(text_pairs[0][0]))
print('source_token_ids:\n', source_token_ids)
print('source_mask:\n', source_mask)
print('target_tokens:\n', tokenizer.tokenize(text_pairs[0][1]))
print('target_token_ids:\n', target_token_ids)
print('target_mask:\n', target_mask)

print('source_token_ids.shape:', source_token_ids.shape)
print('source_mask.shape:', source_mask.shape)
print('target_token_ids.shape:', target_token_ids.shape)
print('target_mask.shape:', target_mask.shape)

source_tokens:
 ['▁Dear', '▁friends', ',', '▁the', '▁history', '▁of', '▁this', '▁nation', '▁includes', '▁many', '▁examples', '▁of', '▁the', '▁Church', '’', 's', '▁commitment', '▁in', '▁this', '▁regard', '.']
source_token_ids:
 tensor([[13959,  1566,    12, 21076,    10, 19451,   803,     6,     8,   892,
            13,    48,  2982,   963,   186,  4062,    13,     8,  2345,    22,
             7,  3148,    16,    48,  3553,     5,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])
source_mask:
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])
target_tokens:
 ['▁Qu', 'er', 'idos', '▁', 'ami', 'go', 's', ',', '▁', 'a', '▁his', 't', 'ó', 'r', 'i', 'a', '▁de', 'sta', '▁Na', 'ç', 'ã', '▁', 'o', '▁of', 'er', 'e', 'ce', '▁numero'

# Etapa NaturalQuestions -> Inferência

## Data prep

In [ ]:
doct_text_df = pd.read_csv(nq_doct_text_path)
long_answer_df = pd.read_csv(nq_long_answer_path)
short_answers_df = pd.read_csv(nq_short_answers_path)
question_df = pd.read_csv(nq_question_path)
infos_df = pd.read_csv(nq_infos_path)

In [ ]:
# doct_text_df = pd.read_csv(nq_doct_text_chunck_path)
# long_answer_df = pd.read_csv(nq_long_answer_chunck_path)
# short_answers_df = pd.read_csv(nq_short_answers_chunck_path)
# question_df = pd.read_csv(nq_question_chunck_path)
# infos_df = pd.read_csv(nq_infos_chunck_path)

In [ ]:
#doct_text_df.head()

In [ ]:
#long_answer_df.head()

In [ ]:
#short_answers_df.head()

In [ ]:
#question_df.head()

In [ ]:
#infos_df.head()

## Classe Data set

In [ ]:
phrase = 'Eu não gostaria de fazer parte desta pandemia'
encoder = tokenizer.encode_plus(phrase,
                      max_length =15,
                      pad_to_max_length=True,
                      add_special_tokens = True)
print('TOKENIZAÇÃO: ',tokenizer.tokenize(phrase))
print('ENCODER: ',encoder)
print('DECODIFICANDO: ', tokenizer.decode(encoder['input_ids']))

TOKENIZAÇÃO:  ['▁Eu', 'não', '▁go', 's', 'tari', 'a', '▁de', '▁', 'f', 'a', 'zer', '▁parte', '▁de', 'sta', '▁pan', 'd', 'emia']
ENCODER:  {'input_ids': [4491, 32100, 281, 7, 5310, 9, 20, 3, 89, 9, 2558, 2359, 20, 2427, 2131], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
DECODIFICANDO:  Eu não gostaria de fazer parte desta pan


In [ ]:
doct_text_df_debug =doct_text_df[:10]
long_answer_df_debug = long_answer_df[:10]
short_answers_df_debug = short_answers_df[:10]
question_df_debug = question_df[:10]

In [ ]:
class QuestionDataset(Dataset):
    def __init__(self, question,tokenizer,max_length,training_step=False):
        self.question = question
        self.max_length = max_length
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.question)

    def __getitem__(self, idx):
        data = self.question.iloc[idx]
        id_example = data['ID']
        question = data['QUESTION']
        q_tok, q_mask, q_type = self.encode_plus(ftfy.fix_text(question))

        return  q_tok, q_mask, q_type, id_example

    def encode_plus(self, text):
        tokens = self.tokenizer.encode_plus(text=text, max_length=self.max_length,
                                       pad_to_max_length=True, add_special_tokens = True)
        tok =  torch.tensor(tokens["input_ids"]).type(torch.long)
        mask = torch.tensor(tokens['attention_mask']).type(torch.long)
        tok_type = torch.tensor(tokens['token_type_ids']).type(torch.long)
        return tok,mask,tok_type

In [ ]:
question_debug = QuestionDataset(
    question = question_df_debug,
    tokenizer=tokenizer,
    max_length=20)

dataloader_debug = DataLoader(question_debug, batch_size=2, shuffle=True,num_workers=cpu_count())

q_tok, q_mask, q_type, id_example = next(iter(dataloader_debug))
print(id_example)
print(q_tok)
print(q_mask)
print(q_type)


tensor([5328212470870865242, 5611750702541347162])
tensor([[  149,     3,    23,     5,  3493,    39,  2039,   113,    19,     8,
          2039,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  113,   410,     8,  2249,    13,     8, 31412,    16,  2515,  3481,
            63,     8,  2983,   348,     0,     0,     0,     0,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
tokenizer.decode(q_tok)

In [ ]:
class LongAnswerDataset(Dataset):
    def __init__(self, long_answer,tokenizer,max_length,training_step=False):
        self.long_answer = long_answer
        self.max_length = max_length
        self.tokenizer = tokenizer

    def __len__(self):
        return 5

    def __getitem__(self, idx):
        data = self.long_answer.iloc[idx]
        id_example = data['ID']
        long_answer = data['LONG_ANSWER']
        start = data['START_LA']
        end = data['END_LA']
        la_tok, la_mask, la_type = self.encode_plus(ftfy.fix_text(long_answer))

        return  la_tok, la_mask, la_type,id_example,start,end

    def encode_plus(self, text):
        tokens = self.tokenizer.encode_plus(text=text, max_length=self.max_length,
                                       pad_to_max_length=True, add_special_tokens = True)
        tok =  torch.tensor(tokens["input_ids"]).type(torch.long)
        mask = torch.tensor(tokens['attention_mask']).type(torch.long)
        tok_type = torch.tensor(tokens['token_type_ids']).type(torch.long)
        return tok,mask,tok_type

In [ ]:
la_debug = LongAnswerDataset(
    long_answer = long_answer_df_debug,
    tokenizer=tokenizer,
    max_length=20)

dataloader_debug = DataLoader(la_debug, batch_size=2, shuffle=True,num_workers=cpu_count())

la_tok, la_mask, la_type,id_example,start,end= next(iter(dataloader_debug))

print(id_example)
print(start)
print(end)
print(la_tok)
print(la_mask)
print(la_type)

tensor([5655493461695504401, 5328212470870865242])
tensor([1718,  132])
tensor([1783,  228])
tensor([[   86,    48,   686,    13,  3662,     3,     6,     3,     9,   349,
            24,  2746,    12,  1299,     3,     9,  7288,    12,    70,   722],
        [29464,  7040,   106, 10361,     3,     6,   394,   801,    38,     3,
             2,    37,  8007,     3,    31,    31,     3,     6,    19,     8]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
class ShortAnswerDataset(Dataset):
    def __init__(self, short_answer,tokenizer,max_length,training_step=False):
        self.short_answer = short_answer
        self.max_length = max_length
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.short_answer)

    def __getitem__(self, idx):
        data = self.short_answer.iloc[idx]
        id_example = data['ID']
        short_answer = data['SHORT_ANSWER']
        start = data['START_SA']
        end = data['END_SA']
        sa_tok, sa_mask, sa_type = self.encode_plus(ftfy.fix_text(short_answer))
        return  sa_tok, sa_mask, sa_type,id_example,start,end

    def encode_plus(self, text):
        tokens = self.tokenizer.encode_plus(text=text, max_length=self.max_length,
                                       pad_to_max_length=True, add_special_tokens = True)
        tok =  torch.tensor(tokens["input_ids"]).type(torch.long)
        mask = torch.tensor(tokens['attention_mask']).type(torch.long)
        tok_type = torch.tensor(tokens['token_type_ids']).type(torch.long)
        return tok,mask,tok_type

In [ ]:
sa_debug = ShortAnswerDataset(
    short_answer = short_answers_df_debug,
    tokenizer=tokenizer,
    max_length=20)

dataloader_debug = DataLoader(sa_debug, batch_size=2, shuffle=True,num_workers=cpu_count())

sa_tok, sa_mask, sa_type,id_example,start,end= next(iter(dataloader_debug))
print(id_example)
print(start)
print(end)
print(sa_tok)
print(sa_mask)
print(sa_type)

tensor([-1446769669686008214,  -341902602485554777])
tensor([3968,  103])
tensor([3993,  107])
tensor([[   46,  3165,  6681,  2535,   113,    19,    44,   709,  1401,   203,
            13,  1246,    42,  2749,    11,    16,     8, 10837,  3143,    13],
        [ 8208,    32, 17457,   325,   354,     2,  2975,   157,    13, 30629,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
class DocTextDataset(Dataset):
    def __init__(self, doc_text,tokenizer,max_length,training_step=False):
        self.doc_text = doc_text
        self.max_length = max_length
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.doc_text)

    def __getitem__(self, idx):
        data = self.doc_text.iloc[idx]
        id_example = data['ID']
        doc_text = data['DOC_TEXT']
        dt_tok, dt_mask, dt_type = self.encode_plus(ftfy.fix_text(doc_text))

        return  dt_tok, dt_mask, dt_type,id_example

    def encode_plus(self, text):
        tokens = self.tokenizer.encode_plus(text=text, max_length=self.max_length,
                                       pad_to_max_length=True, add_special_tokens = True)
        tok =  torch.tensor(tokens["input_ids"]).type(torch.long)
        mask = torch.tensor(tokens['attention_mask']).type(torch.long)
        tok_type = torch.tensor(tokens['token_type_ids']).type(torch.long)
        return tok,mask,tok_type

In [ ]:
doc_debug = DocTextDataset(
    doc_text = doct_text_df_debug,
    tokenizer=tokenizer,
    max_length=20)

dataloader_debug = DataLoader(doc_debug, batch_size=2, shuffle=True,num_workers=cpu_count())

dt_tok, dt_mask, dt_type,id_example= next(iter(dataloader_debug))
print(id_example)
print(dt_tok)
print(dt_mask)
print(dt_type)

tensor([5655493461695504401, 5655493461695504401])
tensor([[  597, 15551,  1037,   164,    36, 15514,    11,  3641,     3,     5,
            41,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [ 4001,   149,    11,   116,    12,  2036,   175,  3847,  4175,     3,
            61,   100,  1108,   523,  1151,     3, 13903,     7,    21, 17549]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


# Hyperparâmetros

In [ ]:
classification_specific = False #@param {type:"boolean"}
batch_size =  32#@param {type:"integer"}
max_epochs = 2 #@param {type:"integer"}
accumulate_grad_batches = 8  #@param {type:"integer"}
source_max_length = 64  #@param {type:"integer"}
target_max_length = 128  #@param {type:"integer"}
max_lenght_NQ = 500  #@param {type:"integer"}
learning_rate = 5e-3  #@param {type:"number"}
context_size = 20

In [ ]:
hyperparms = {'model_name':model_name,'tokenizer':tokenizer,'learning_rate':learning_rate,'batch_size':batch_size,'source_max_length':source_max_length,'target_max_length':target_max_length,'context_size':context_size,'max_lenght_NQ':max_lenght_NQ} 

## Criando o BERT com Pytorch Lightning

In [ ]:
class T5Finetuner(pl.LightningModule):

    def __init__(self, 
                 all_data, 
                 hyperparms,
                 criterion = torch.nn.CrossEntropyLoss(),
                 overfit=False):
      

        super(T5Finetuner, self).__init__()

        #---------- Hyperparametros
        self.model_name = hyperparms['model_name']
        self.tokenizer = hyperparms['tokenizer']
        self.learning_rate = hyperparms['learning_rate']
        self.batch_size = hyperparms['batch_size']
        self.target_max_length = hyperparms['target_max_length']
        self.source_max_length = hyperparms['source_max_length']
        self.max_lenght_NQ = hyperparms['max_lenght_NQ']
        self.overfit = overfit
        self.training = False

      
        self.model = T5ForConditionalGeneration.from_pretrained(self.model_name)

       #---------- Carregamento datasets (Para eu poder variar self.max_length)
        if overfit:
          self.train_dataset = ParacrawlDataset(all_data[0], tokenizer=self.tokenizer,source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
          self.train_dataset = ParacrawlDataset(all_data[0], tokenizer=self.tokenizer,source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
          self.train_dataset = ParacrawlDataset(all_data[0], tokenizer=self.tokenizer,source_max_length = self.source_max_length, target_max_length =  self.target_max_length)          
          # self.valid_dataset = ParacrawlDataset(all_data[0], tokenizer=self.tokenizer, source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
          # self.test_dataset =   ParacrawlDataset(all_data[0], tokenizer=self.tokenizer,source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
        else:
          self.train_dataset = ParacrawlDataset(all_data[0], tokenizer=self.tokenizer, source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
          self.valid_dataset = ParacrawlDataset(all_data[1], tokenizer=self.tokenizer,source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
          self.test_dataset = ParacrawlDataset(all_data[2], tokenizer=self.tokenizer,source_max_length = self.source_max_length, target_max_length =  self.target_max_length)

          #self.valid_dataset = QuestionDataset(all_data[1], tokenizer=self.tokenizer, max_lenght_NQ =  self.max_lenght_NQ)
          #self.test_dataset = QuestionDataset(all_data[2],  tokenizer=self.tokenizer, max_lenght_NQ =  self.max_lenght_NQ)
        
        
        #---------- Loss Function
        self.loss_funtion = criterion

    

    def translate(self, source_token_ids, source_mask):
      predicted_token_ids = self.model.generate(input_ids=source_token_ids, max_length=self.target_max_length)
      return predicted_token_ids

    def forward(self, source_token_ids, source_mask, target_token_ids=None,target_mask=None, training=False):

      if training:
          #peguei a ideia da documentação oficial. Setar -100 pro pad faz ele ser ignorado no crossentropy.
          target_token_ids[target_token_ids == self.tokenizer.pad_token_id] = -100 
          
          loss = self.model(source_token_ids, attention_mask = source_mask, lm_labels = target_token_ids)
          return loss[0]

      else:
          #gerador de tokens de saída - GREEDY
          predicted_token_ids = self.model.generate(input_ids=source_token_ids, max_length=self.target_max_length)
          return predicted_token_ids


    def training_step(self, batch, batch_nb):
        # batch
        source_token_ids, source_mask, target_token_ids, target_mask, _, _ = batch
         
        # fwd
        loss = self.forward(source_token_ids, source_mask, target_token_ids, target_mask, training=True)

        # logs
        tensorboard_logs = {'train_loss': loss}
        progress_bar = {'gpu_usage': gpu_usage()}
        return {'loss': loss, 'log': tensorboard_logs,
                'progress_bar': progress_bar}


    def validation_step(self, batch, batch_nb):
        source_token_ids, source_mask, target_token_ids, target_mask, source, refs = batch
        predict = self(source_token_ids, source_mask).permute(0,1)
        sys = [fix_accent_breaks(self.tokenizer.decode(tokens)) for tokens in predict]
        avg_bleu = sacrebleu.corpus_bleu(sys, [refs]).score
        progress_bar = {'gpu_usage': gpu_usage()}
        return {'val_bleu': avg_bleu, 'progress_bar': progress_bar}


    def test_step(self, batch, batch_nb):
        source_token_ids, source_mask, target_token_ids, target_mask, source, refs = batch
        predict = self(source_token_ids, source_mask).permute(0,1)
        sys = [fix_accent_breaks(self.tokenizer.decode(tokens)) for tokens in predict]
        
        avg_bleu = sacrebleu.corpus_bleu(sys, [refs]).score
        
        progress_bar = {'gpu_usage': gpu_usage()}
        return {'test_bleu': avg_bleu, 'progress_bar': progress_bar}

    def validation_epoch_end(self, outputs):
        avg_bleu = sum([x['val_bleu'] for x in outputs]) / len(outputs)
        print("Avg Bleu val", avg_bleu)
        tensorboard_logs = {'avg_val_bleu': avg_bleu}
        
        return {'avg_val_bleu': avg_bleu, 'progress_bar': tensorboard_logs}


    def test_epoch_end(self, outputs):
        avg_bleu = sum([x['test_bleu'] for x in outputs]) / len(outputs)

        tensorboard_logs = {'avg_test_bleu': avg_bleu}

    def configure_optimizers(self):
        return torch.optim.AdamW(
            [p for p in self.parameters() if p.requires_grad],
            lr=self.learning_rate, eps=1e-08)
    
    @gpu_mem_restore
    def train_dataloader(self):
        shuffle = False if self.overfit else True
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=shuffle,num_workers=cpu_count())
    
    @gpu_mem_restore
    def val_dataloader(self):
        return DataLoader(self.valid_dataset, batch_size=self.batch_size, shuffle=False,num_workers=cpu_count())
    
    @gpu_mem_restore
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size,shuffle=False, num_workers=cpu_count())

# Etapas de Treino e Avaliação

### Testando rapidamente o modelo em treino, validação e teste com um batch

Recuperando o número de parâmetros

In [ ]:
all_data = [ds_train,ds_val,ds_test]
model = T5Finetuner(all_data,hyperparms) 
num_params = sum([torch.tensor(x.size()).prod() for x in model.parameters() if x.requires_grad]) # trainable parameters
print(num_params)

tensor(60506880)


In [ ]:
trainer = pl.Trainer(gpus=1, 
                     checkpoint_callback=False,  # Disable checkpoint saving.
                     fast_dev_run=True)
trainer.fit(model)

No environment variable for node rank defined. Set as 0.
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Avg Bleu val 0.7478088493046045



1

In [ ]:
del model

In [ ]:
check_path = '/content/drive/My Drive/Projeto/Data/Traducao'
checkpoint_path = check_path + '/Checkpoints_V2/epoch=15.ckpt'
print(checkpoint_path)

### Treinamento e Validação no dataset todo

In [ ]:
# checkpoint_path = check_path + '/epoch=0.ckpt'

checkpoint_path = check_path + '/Checkpoints_V2/epoch=15.ckpt'
check_path = '/content/drive/My Drive/Projeto/Data/Traducao'
checkpoint_dir = os.path.dirname(os.path.abspath(checkpoint_path))
print(f'Files in {checkpoint_dir}: {os.listdir(checkpoint_dir)}')
print(f'Saving checkpoints to {checkpoint_dir}')
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_dir,
                                      save_top_k=-1,
                                      monitor="val_acc",
                                      mode="max")  # Keeps all checkpoints.


max_epochs = 14

checkpoint_path = '/content/drive/My Drive/Alexandre_aula_10/epoch=13.ckpt'
checkpoint_dir = os.path.dirname(os.path.abspath(checkpoint_path))
print(f'Files in {checkpoint_dir}: {os.listdir(checkpoint_dir)}')
print(f'Saving checkpoints to {checkpoint_dir}')
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_dir,
                                      save_top_k=-1)  # Keeps all checkpoints.

resume_from_checkpoint = None
if os.path.exists(checkpoint_path):
    print(f'Restoring checkpoint: {checkpoint_path}')
    resume_from_checkpoint = checkpoint_path


all_data = [ds_train,ds_val,ds_test]
trainer = pl.Trainer(gpus=1,
                     max_epochs=15,
                     check_val_every_n_epoch=1,
                     profiler=True,
                     checkpoint_callback=checkpoint_callback,
                     accumulate_grad_batches=accumulate_grad_batches,
                     progress_bar_refresh_rate=50,
                     resume_from_checkpoint=resume_from_checkpoint)

model = T5Finetuner(all_data,hyperparms) 

trainer.fit(model)


No environment variable for node rank defined. Set as 0.


Files in /content/drive/My Drive/Projeto/Data/Traducao/Checkpoints_V2: ['epoch=0.ckpt', 'epoch=0_v0.ckpt', 'epoch=1.ckpt', 'epoch=2.ckpt', 'epoch=3.ckpt', 'epoch=4.ckpt', 'epoch=5.ckpt', 'epoch=6.ckpt', 'epoch=7.ckpt', 'epoch=8.ckpt', 'epoch=9.ckpt', 'epoch=10.ckpt', 'epoch=11.ckpt', 'epoch=12.ckpt', 'epoch=13.ckpt', 'epoch=14.ckpt']
Saving checkpoints to /content/drive/My Drive/Projeto/Data/Traducao/Checkpoints_V2


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Avg Bleu val 6.0780716540512225


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1

## Colocando no formato do MS MARCO

### Document

In [ ]:
# checkpoint_path = check_path + '/epoch=14.ckpt'
# model = T5Finetuner.load_from_checkpoint(checkpoint_path = checkpoint_path,
#                                          all_data = all_data,
#                                          hyperparms=hyperparms).to(device)

NameError: ignored

In [ ]:
doc_text_dataset = DocTextDataset(
    doc_text = doct_text_df,
    tokenizer=tokenizer,
    max_length=max_lenght_NQ)

doc_text_dataloader = DataLoader(doc_text_dataset, batch_size=batch_size,num_workers=cpu_count())

In [ ]:
ids = []
doc_text_portuguese = []

for batch in doc_text_dataloader:
    #ids
    ids.extend(batch[3].detach().cpu().tolist())    
    #doc_text
    out = model(batch[0].to(device), batch[1].to(device))
    portuguese_translations = [fix_accent_breaks(tokenizer.decode(tokens)) for tokens in out]
    doc_text_portuguese.extend(portuguese_translations)


In [ ]:
doct_text_df_portuguese = pd.DataFrame({'ID':ids, 'DOC_TEXT': doc_text_portuguese})

In [ ]:
doct_text_df_portuguese.head()

In [ ]:
compression_opts = dict(method='zip', archive_name='doct_text_df_portuguese.csv') 
doct_text_df_portuguese.to_csv('doct_text_df_portuguese.zip', index=False,compression=compression_opts)

### Question

In [ ]:
question_dataset = QuestionDataset(
    question = question_df,
    tokenizer=tokenizer,
    max_length=max_lenght_NQ)

question_dataloader = DataLoader(question_dataset, batch_size=batch_size,num_workers=cpu_count())

In [ ]:
ids = []
question_portuguese = []


for batch in question_dataloader:
    #ids
    ids.extend(batch[3].detach().cpu().tolist())    
    #question
    out = model(batch[0].to(device), batch[1].to(device))
    portuguese_translations = [fix_accent_breaks(tokenizer.decode(tokens)) for tokens in out]
    question_portuguese.extend(portuguese_translations)


In [ ]:
question_df_portuguese = pd.DataFrame({'ID': ids, 'QUESTION': question_portuguese})

In [ ]:
question_df_portuguese.head()

In [ ]:
compression_opts = dict(method='zip', archive_name='question_df_portuguese.csv') 
question_df_portuguese.to_csv('question_df_portuguese.zip', index=False,compression=compression_opts)

### Long Answer

In [ ]:
long_answer_dataset = LongAnswerDataset(
    long_answer = long_answer_df,
    tokenizer=tokenizer,
    max_length=max_lenght_NQ)

long_answer_dataloader = DataLoader(long_answer_dataset, batch_size=batch_size,num_workers=cpu_count())

In [ ]:
ids = []
long_answer_portuguese = []
start_token = []
end_token = []

for batch in long_answer_dataloader:
    #ids
    ids.extend(batch[3].detach().cpu().tolist()) 
    # Start token
    start_token.extend(batch[4].detach().cpu().tolist()) 
    # End token
    end_token.extend(batch[5].detach().cpu().tolist()) 
    #question
    out = model(batch[0].to(device), batch[1].to(device))
    portuguese_translations = [fix_accent_breaks(tokenizer.decode(tokens)) for tokens in out]
    long_answer_portuguese.extend(portuguese_translations)
    

In [ ]:
long_answer_df_portuguese = pd.DataFrame({'ID': ids, 'LONG_ANSWER': long_answer_portuguese, 'START_LA': start_token,'END_LA':end_token})

In [ ]:
long_answer_df_portuguese.head()

In [ ]:
compression_opts = dict(method='zip', archive_name='long_answer_df_portuguese.csv') 
long_answer_df_portuguese.to_csv('long_answer_df_portuguese.zip', index=False,compression=compression_opts)

### Short Answer

In [ ]:
short_answer_dataset = ShortAnswerDataset(
    short_answer = short_answers_df,
    tokenizer=tokenizer,
    max_length=max_lenght_NQ)

short_answer_dataloader = DataLoader(short_answer_dataset, batch_size=batch_size,num_workers=cpu_count())

In [ ]:
ids = []
short_answer_portuguese = []
start_token = []
end_token = []

for batch in short_answer_dataloader:
    #ids
    ids.extend(batch[3].detach().cpu().tolist()) 
    # Start token
    start_token.extend(batch[4].detach().cpu().tolist()) 
    # End token
    end_token.extend(batch[5].detach().cpu().tolist()) 
    #question
    out = model(batch[0].to(device), batch[1].to(device))
    long_answer_portuguese = [fix_accent_breaks(tokenizer.decode(tokens)) for tokens in out]
    short_answer_portuguese.extend(long_answer_portuguese)

In [ ]:
short_answer_df_portuguese = pd.DataFrame({'ID': ids, 'SHORT_ANSWER': short_answer_portuguese, 'START_SA': start_token,'END_SA':end_token})

In [ ]:
short_answer_df_portuguese.head()

In [ ]:
compression_opts = dict(method='zip', archive_name='short_answer_df_portuguese.csv') 
short_answer_df_portuguese.to_csv('short_answer_df_portuguese.zip', index=False,compression=compression_opts)